codecs 模块提供了流和文件接口用于转换数据。它通常用于处理 Unicode 文本。

In [2]:
import binascii

def to_hex(t, nbytes):
    """Format text t as a sequence of nbyte long values separated by spaces"""
    chars_per_item = nbytes*2
    hex_version = binascii.hexlify(t)
    return b' '.join(hex_version[start:start + chars_per_item] for start in range(0, len(hex_version), chars_per_item))

In [3]:
print(to_hex(b'abcdef', 1))
print(to_hex(b'abcdef', 2))

b'61 62 63 64 65 66'
b'6162 6364 6566'


In [8]:
import unicodedata

text = 'françaiS'
print(f'Raw: {text!r}')
for c in text:
    print(f"  {c!r}:{unicodedata.name(c,c)}")
    
print(f"UTF-8 : {to_hex(text.encode('utf-8'),1)}")
print(f"UTF-16 : {to_hex(text.encode('utf-16'),1)}")
print(f"UTF-16 : {to_hex(text.encode('utf-16'),2)}")

Raw: 'françaiS'
  'f':LATIN SMALL LETTER F
  'r':LATIN SMALL LETTER R
  'a':LATIN SMALL LETTER A
  'n':LATIN SMALL LETTER N
  'ç':LATIN SMALL LETTER C WITH CEDILLA
  'a':LATIN SMALL LETTER A
  'i':LATIN SMALL LETTER I
  'S':LATIN CAPITAL LETTER S
UTF-8 : b'66 72 61 6e c3 a7 61 69 53'
UTF-16 : b'ff fe 66 00 72 00 61 00 6e 00 e7 00 61 00 69 00 53 00'
UTF-16 : b'fffe 6600 7200 6100 6e00 e700 6100 6900 5300'
